# Load enviroment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Clone to Google Drive
# !git clone https://github.com/guilherme-webster/NavierStokes-parallel.git /content/gdrive/MyDrive/NavierStokes-parallel

In [ ]:
import subprocess
import os
import numpy as np
base_dir = "/content/gdrive/MyDrive/NavierStokes-parallel"
get_ipython().run_line_magic('cd', base_dir)

# Aux functions declarations

In [ ]:
# Helper functions to compare CFD simulation outputs
def extract_center_values(filename):
    """Extract the center U, V, and P values from the output file"""
    values = {"u_center": [], "v_center": [], "p_center": []}
    
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            
        for i, line in enumerate(lines):
            if "U-CENTER:" in line:
                try:
                    value = float(line.split(":")[-1].strip())
                    if not np.isnan(value) and not np.isinf(value):
                        values["u_center"].append(value)
                except ValueError:
                    continue
                    
            elif "V-CENTER:" in line:
                try:
                    value = float(line.split(":")[-1].strip())
                    if not np.isnan(value) and not np.isinf(value):
                        values["v_center"].append(value)
                except ValueError:
                    continue
                    
            elif "P-CENTER:" in line:
                try:
                    value = float(line.split(":")[-1].strip())
                    if not np.isnan(value) and not np.isinf(value):
                        values["p_center"].append(value)
                except ValueError:
                    continue
    except Exception as e:
        print(f"Error extracting values from {filename}: {e}")
        
    return values

def compare_cfd_outputs(serial_file, parallel_file, relative_tolerance=0.05):
    """Compare CFD outputs with relative tolerance for numerical differences"""
    serial_values = extract_center_values(serial_file)
    parallel_values = extract_center_values(parallel_file)
    
    # Make sure we have values to compare
    if not all(serial_values.values()) or not all(parallel_values.values()):
        return False, "Missing values in output files"
    
    # Compare final state (last 10% of timesteps)
    errors = []
    
    for key in ["u_center", "v_center", "p_center"]:
        # Get last 10% of values or at least the last value
        serial_final = serial_values[key][-max(1, len(serial_values[key])//10):]
        parallel_final = parallel_values[key][-max(1, len(parallel_values[key])//10):]
        
        # If different number of timesteps, just compare the last value
        if len(serial_final) != len(parallel_final):
            serial_final = [serial_final[-1]]
            parallel_final = [parallel_final[-1]]
        
        for s_val, p_val in zip(serial_final, parallel_final):
            # Avoid division by zero with small denominator
            denominator = max(abs(s_val), 1e-6)
            rel_error = abs(s_val - p_val) / denominator
            errors.append(rel_error)
    
    max_error = max(errors) if errors else float('inf')
    avg_error = sum(errors)/len(errors) if errors else float('inf')
    
    if max_error <= relative_tolerance:
        return True, f"Outputs match within tolerance. Avg error: {avg_error:.6f}, Max error: {max_error:.6f}"
    else:
        return False, f"Outputs differ beyond tolerance. Avg error: {avg_error:.6f}, Max error: {max_error:.6f}"

# Running and comparing with serial

In [ ]:
# set exercise path
exercise_dir = base_dir

# Function to run shell commands in a specific directory
def run_command(command, work_dir):
    return subprocess.run(command, shell=True, cwd=work_dir, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Create and prepare the build directory
cmd_return = run_command('cmake -E remove -f build', exercise_dir)
print(f"Cmake output:\n{cmd_return.stdout}")

if cmd_return.returncode != 0:
    print(f"Cmake failed with error:\n{cmd_return.stderr}")
    assert False

run_command('cmake -E make_directory build', exercise_dir)
cmd_return = run_command('cmake ../ -DCMAKE_BUILD_TYPE=Release', os.path.join(exercise_dir, 'build'))
print(f"Cmake output:\n{cmd_return.stdout}")

if cmd_return.returncode != 0:
    print(f"Cmake failed with error:\n{cmd_return.stderr}")
    assert False

# Compile the programs
cmd_return = run_command('make', os.path.join(exercise_dir, 'build'))
print(f"Compilation output:\n{cmd_return.stdout}")

if cmd_return.returncode != 0:
    print(f"Compilation failed with error:\n{cmd_return.stderr}")
    assert False

# Execute tests and collect outputs
for i in range(1, 6):
    test_input = f"tests/{i}.in"

    if not os.path.exists(os.path.join(exercise_dir, test_input)):
        printf("Error! Test input not found")

    parallel_output = f"parallel.{i}.out"
    serial_output = f"serial.{i}.out"
    parallel_time = f"parallel.{i}.time"
    serial_time = f"serial.{i}.time"


    # Workaround for GDrive permissions
    subprocess.run(f"chmod 755 ./parallel", shell=True, cwd=os.path.join(exercise_dir, 'build'))
    subprocess.run(f"chmod 755 ./serial", shell=True, cwd=os.path.join(exercise_dir, 'build'))

    # Run parallel and serial programs
    subprocess.run(f"build/parallel {test_input} > build/{parallel_output} 2> build/{parallel_time}", shell=True, cwd=os.path.join(exercise_dir))
    subprocess.run(f"build/serial {test_input} > build/{serial_output} 2> build/{serial_time}", shell=True, cwd=os.path.join(exercise_dir))

    # Compare outputs and calculate speedup
    diff_result = subprocess.run(f"cmp {serial_output} {parallel_output}", shell=True, cwd=os.path.join(exercise_dir, 'build'), stderr=subprocess.PIPE)
    diff_status = diff_result.returncode

    # Read timings
    with open(os.path.join(exercise_dir, 'build', serial_time), 'r') as f:
        serial_runtime = float(f.read().strip())
    
    try:
        with open(os.path.join(exercise_dir, 'build', parallel_time), 'r') as f:
            parallel_runtime = float(f.read().strip())
    except:
        parallel_runtime = 0

    # Compare outputs with relaxed tolerance appropriate for CFD
    match, message = compare_cfd_outputs(
        os.path.join(exercise_dir, 'build', serial_output),
        os.path.join(exercise_dir, 'build', parallel_output),
        relative_tolerance=0.05  # 5% relative tolerance
    )
    
    test_status = "[CORRECT OUTPUT]" if match else "[APPROXIMATE OUTPUT]" if "Avg error" in message and float(message.split("Avg error:")[1].split(",")[0]) < 0.2 else "[INCORRECT OUTPUT]"

    # Calculate and display speedup
    if match or "Avg error" in message:
        speedup = serial_runtime / parallel_runtime if parallel_runtime > 0 else float('inf')
        print(f"Test {i}: {test_status}")
        print(f"  Serial time = {serial_runtime:.6f}s")
        print(f"  Parallel time = {parallel_runtime:.6f}s")
        print(f"  Speedup = {speedup:.4f}x")
        print(f"  {message}\n")
    else:
        print(f"Test {i}: {test_status}")
        print(f"  Serial time = {serial_runtime:.6f}s")
        print(f"  {message}\n")